### TrumpTweet Notebook
This notebook references the code to create a Trump tweet. This includes:
- Processing the Trump Tweet archive to create a clean file of Trump's tweets
- Defining and training GRU long short term memory recurrent neural network
- Feeding data into the model in order to create a novel tweet

#### Data Prep

In [1]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Run the helpers scripts with the data and model helper objects
%run scripts/helpers
#from scripts import helpers

In [3]:
# Verify GPU support
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

##### Create a datahelper and process raw data

In [4]:
# Create a datahelper object and designate the input file
dh = DataHelper(file_name='tweets_11-06-2020.csv')

# Prep the raw data to create the tweet file
dh.prep_raw_data(start_date='2020-08-01', end_date='2020-11-19')

# Print the number of tweets
print('The number of Tweets sent by Trump during the period is {}'.format(dh.num_tweets))


Data processing complete.
The number of Tweets sent by Trump during the period is 2362


##### Tokenize the text and create the dataset for model training

In [5]:
# Tokenize the text and create the dataset
dataset, tokenizer = dh.create_tokenizer('inputdata/clean_tweet.txt')
print('The number of unique characters is {0:,} and the dataset size is {1:,} document(s).' \
      ' The number of windows in the dataset for processing is {2:,}.'.format(dh.num_unique_chars,dh.dataset_size,dh.num_data_windows))


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '__dict__'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '__dict__'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '__dict__'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosi

##### Create the Model and Train It
The model is a stateless RNN made up of GRU cells

In [ ]:
# Create the modelhelper object
mh = ModelHelper(epochs=20)

# Create the model
model = mh.create_model(tokenizer)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam')

# Save a checkpoint after every epoch
EPOCHS = 20
checkpoint_filepath = 'checkpoints/weights.{epoch:02d}.hdf5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    save_freq='epoch',
    monitor='val_loss',
    mode='min',
    save_best_only=False)

# Fit the model
history = model.fit(dataset,epochs=EPOCHS,callbacks=[model_checkpoint])


Epoch 1/20
136851/136851 [==============================] - 10397s 76ms/step - loss: 1.3428
Epoch 2/20
136851/136851 [==============================] - 10362s 76ms/step - loss: 1.1128
Epoch 3/20
136851/136851 [==============================] - 10386s 76ms/step - loss: 1.0804
Epoch 4/20
136851/136851 [==============================] - 10365s 76ms/step - loss: 1.0646
Epoch 5/20
136851/136851 [==============================] - 10365s 76ms/step - loss: 1.0524
Epoch 6/20
136851/136851 [==============================] - 10364s 76ms/step - loss: 1.0425
Epoch 7/20
 83734/136851 [=================>............] - ETA: 1:07:00 - loss: 1.0354

##### Start Training Model from Saved Checkpoint
Training the model takes quite some time. A checkpoint is saved every epoch so the code below will allow you to resume training from a checkpoint

In [ ]:
#restart training from saved checkpoint
new_model = load_model('checkpoints/weights.20.model1.hdf5')

# Save a checkpoint after every epoch
EPOCHS = 20
checkpoint_filepath = 'checkpoints/weights.{epoch:02d}.hdf5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    save_freq='epoch',
    monitor='val_loss',
    mode='min',
    save_best_only=False)

#Fit the model
history = new_model.fit(dataset,epochs=EPOCHS,callbacks=[model_checkpoint])


In [ ]:
# Save model
model.save('model.h5')

##### Generate some tweets

In [ ]:
# Create a long sequence of text
print(mh.create_tweet('election', temperature=0.2))